In [1]:
import pandas as pd
import numpy as np


In [2]:
df1 = pd.read_csv('rte500.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)



C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df1.label[df1.label == 1]="pos"
df1.label[df1.label == 0]="neg"


C:\Users\moshi\AppData\Local\Temp/ipykernel_2844/321102683.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.label[df1.label == 1]="pos"
C:\Users\moshi\AppData\Local\Temp/ipykernel_2844/321102683.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.label[df1.label == 0]="neg"


In [4]:

temp = df1

In [5]:
temp["label"].value_counts()

neg    250
pos    250
Name: label, dtype: int64

In [6]:
temp.head()

,Unnamed: 0,sentence1,sentence2,label
0,485,"Rodriguez, a four-year NYPD officer, suffered ...",Rodriguez died of burns to his legs.,neg
1,757,North Korean refugees had been gathering in so...,Ho Chi Minh City is now called Saigon.,neg
2,438,These parties are the True Path Party (rightis...,"The True Path Party, the Virtue Party and the ...",neg
3,566,Lyon is actually the gastronomic capital of Fr...,Lyon is the capital of France.,neg
4,52,Prime Minister Allawi has sent out letters to ...,Prime Minister Allawi will return to Iraq in t...,neg


In [7]:
temp.tail()

,Unnamed: 0,sentence1,sentence2,label
495,440,It is to be mentioned that the airport is capa...,"The airport, with a projected annual passenger...",pos
496,568,"The Republic of Yemen is an Arab, Islamic and ...",The national language of Yemen is Arabic.,pos
497,411,Economic experts were surprised by this coordi...,Two main banks reduced their interest rates.,pos
498,322,The debacle marked a new low in the erosion of...,Schroeder was elected in 1998.,pos
499,340,Teenage sensation Wayne Rooney powered England...,England will play the quarter-finals of Euro 2...,pos


In [8]:
temp["label"].value_counts()

neg    250
pos    250
Name: label, dtype: int64

In [9]:
text1 = temp['sentence1']+temp['sentence2']
emotion= temp['label'].tolist()

In [10]:
text=text1.values.tolist()

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text , emotion, test_size=.2, random_state=42)

In [12]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-multilingual-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500)


In [13]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 35
	95percentile : 63
	99percentile : 82


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 37
	95percentile : 61
	99percentile : 70


In [14]:
model = t.get_classifier()

In [15]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [16]:
learner.fit_onecycle(2e-5, 5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
67/67 [==============================] - 889s 13s/step - loss: 0.7025 - accuracy: 0.5050 - val_loss: 0.6823 - val_accuracy: 0.5600
Epoch 2/5
67/67 [==============================] - 845s 13s/step - loss: 0.6717 - accuracy: 0.5975 - val_loss: 0.6171 - val_accuracy: 0.6500
Epoch 3/5
67/67 [==============================] - 826s 12s/step - loss: 0.5878 - accuracy: 0.7175 - val_loss: 0.5728 - val_accuracy: 0.7200
Epoch 4/5
67/67 [==============================] - 836s 12s/step - loss: 0.4142 - accuracy: 0.8350 - val_loss: 0.6507 - val_accuracy: 0.6700
Epoch 5/5
67/67 [==============================] - 868s 13s/step - loss: 0.2487 - accuracy: 0.9225 - val_loss: 0.6134 - val_accuracy: 0.7100


In [17]:
class_names=["TRUE","FALSE"]

In [18]:
learner.validate(val_data=(val), class_names=class_names)

              precision    recall  f1-score   support

        TRUE       0.69      0.67      0.68        46
       FALSE       0.73      0.74      0.73        54

    accuracy                           0.71       100
   macro avg       0.71      0.71      0.71       100
weighted avg       0.71      0.71      0.71       100



array([[31, 15],
       [14, 40]], dtype=int64)